# Задание 1 (4 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

## Решение:

*NOTE: я не знаю, как умудрился получить такие результаты. Прошу прощения за полотна дублированного кода.*

*NOTE2: прогонял заново на нормальном компьютере и со скоростью всё хорошо, но блок с оптимизацией решил оставить*

### Загрузка датасета


In [6]:
import pathlib

corpus_path = pathlib.Path(r"data/corpus_wsd_50k.txt")

with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.read().split('\n\n')
    corpus = map(
        lambda sent: [
            word.split("\t")
            for word
            in sent.split("\n")
            ],
        corpus
        )
    corpus = tuple(corpus)

###  Загрузка ворднета

In [14]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import wordnet as wn
    

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/smertlove/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/smertlove/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


###  Функция, составляющая контекст

Конетекстом считаем предложение, в котором встречается слово, без самого слова. Будем брать предложение из датасета целиком и маскировать в нем слово, для которого нужно определить значение. 

In [15]:
def get_context_mask_target(corpus_sentence:list, target_ind:int) -> list:

    result = list()

    for ind, word in enumerate(corpus_sentence):
        if ind != target_ind:
            result.append(word[1])
        else:
            result.append("_")

    return result

### Препроц

In [16]:
from string import punctuation


class Preprocessor:

    stopwords:list[str] = nltk.corpus.stopwords.words("english")
    stopwords.extend(punctuation)
    stopwords.append("_")
    stopwords = set(stopwords)

    @classmethod
    def tokenize(cls, text:str) -> list[str]:
        return nltk.word_tokenize(text)
    
    @classmethod
    def rm_stopwords(cls, tokenized_text:list[str]) -> list[str]:
        return [c for c in tokenized_text if c not in cls.stopwords]

    @classmethod
    def preprocess(cls, text: str | list[str]) -> list[str]:
        
        if isinstance(text, str):
            text = cls.tokenize(text)
        
        text = [c.lower() for c in text]      
        text = cls.rm_stopwords(text)

        return text

In [17]:
#  Проверяем препроц

s = corpus[1]
s = get_context_mask_target(s, 8)
s = Preprocessor.preprocess(s)
s

['permit',
 'become',
 'giveaway',
 'rather',
 'one',
 'goal',
 'improved',
 'employee',
 'morale',
 'consequently',
 'increased',
 'productivity']

###  Алгоритм Леска в общем виде

In [18]:
class Lesk:

    #  Псевдометрика, переопределяется в классах-наследниках
    metric = lambda sent1, sent2: 0

    @classmethod
    def _get_all_definitions(cls, word:str) -> str:
        return [c.definition() for c in wn.synsets(word)]


    @classmethod
    def get_definition(cls, word:str, context:list[str]) -> str:

        all_results = [
            {
                "definition": definition,
                "score"     : cls.metric(context, Preprocessor.preprocess(definition))
            }
            for definition
            in cls._get_all_definitions(word)
        ]

        return max(all_results, key=lambda c: c["score"])


### Алгоритм Леска с метрикой Жаккара

In [19]:
def jaccard(sent1:list[str], sent2:list[str]) -> float:
    sent1 = set(sent1)
    sent2 = set(sent2)

    return len(sent1 & sent2) / len(sent1 | sent2)


class LeskWithJaccard(Lesk):
    metric = jaccard

In [20]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'score': 0.3}

In [21]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'a motor vehicle with four wheels; usually propelled by an internal combustion engine',
 'score': 0.0}

### Алгоритм Леска с косинусной близостью на эмбедингах sentence_transformers

In [22]:
from sklearn.metrics.pairwise import cosine_distances
from sentence_transformers import SentenceTransformer

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [23]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:

def cos_sim(sent1:list[str], sent2:list[str]) -> float:
    emb1 = model.encode(" ".join(sent1)).reshape(1, -1)
    emb2 = model.encode(" ".join(sent2)).reshape(1, -1)

    return float(cosine_distances(emb1, emb2)[0][0])

class LeskWithCosSim(Lesk):
    metric = cos_sim

In [25]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'where passengers ride up and down',
 'score': 0.8515398502349854}

In [26]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'a wheeled vehicle adapted to the rails of railroad',
 'score': 0.9286145567893982}

### Всё готово для сравнительного тестирования

Осталось пройти по всему датасету, предсказать значения для всех многозначных слов обоими способами и сравнить результаты с эталонными.

In [27]:
from tqdm import tqdm

In [28]:
results = dict()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():
            continue

        else:

            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                context
            )

            cos_sim_result = LeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )


100%|██████████| 10/10 [00:07<00:00,  1.29it/s]


Первые 10 предложений в корпусе обрабатываются 4 с лишним минуты. Что вдвойне обидно, если принять во внимание тот факт, что в коде выше я забыл записать результаты замеров. Тем не менее, есть подозрение, что функция расчета косинусного подобия работает слишком медленно. Ниже попробую оптимизировать ее:

In [29]:

import numpy as np

##  Тут поменяем интерфейс так, чтобы не было лишних операций str.join
##  и заменим функцию расчета косинусной близости, чтобы модель кодировала контекст только 1 раз.
def better_cos_sim(context_sent:str, other_sents:list[str]) -> np.ndarray:
    context_emb = model.encode(context_sent).reshape(1, -1)
    other_embs = [
        model.encode(sent)
        for sent
        in other_sents
    ]
    return cosine_distances(context_emb, other_embs)[0]


##  Тут поменяем логику под новый расчет метрики
class BetterLeskWithCosSim(Lesk):
    
    metric = better_cos_sim

    @classmethod
    def get_definition(cls, word:str, context:str) -> str:

        definitions = cls._get_all_definitions(word)
        metrics: np.ndarray = cls.metric(context, definitions)

        argmax = metrics.argmax()

        return {
            "definition": definitions[argmax],
            "score"     : float(metrics[argmax])
        }


In [30]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = " ".join(Preprocessor.preprocess(ctx))

BetterLeskWithCosSim.get_definition(word, ctx)

{'definition': 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'score': 0.7972331047058105}

In [31]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = " ".join(Preprocessor.preprocess(ctx))

BetterLeskWithCosSim.get_definition(word, ctx)

{'definition': 'a conveyance for passengers or freight on a cable railway',
 'score': 0.9488926529884338}

Результаты другие. Возможно, изначально метрика была посчитана неверно. В любом случае пока что кажется, что имеется неплохое ускорение. Ниже дублирую код замеров, не забыв при этом записать результаты:

In [33]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)
            context = Preprocessor.preprocess(context)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                context
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


Четырехкратное ускорение, это победа. Отпечатаем результаты:

In [34]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    cos_sim_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, cos_sim_result = result

        jaccard_positive_count += jaccard_result == gold
        cos_sim_positive_count += cos_sim_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim % : {cos_sim_positive_count * 100 / total_count}")

In [35]:
count_results(results)

jaccard % : 56.71641791044776
cos_sim % : 14.925373134328359


Ускорение получили, а вот нормально работать в сделку уже не входило. При чем если с жаккаром такие результаты еще можно понять, то с косинусным подобием точно что-то пошло не так. Попробуем убрать препроц в расчете метрики косинусного подобия:

In [36]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

count_results(results)

100%|██████████| 10/10 [00:03<00:00,  3.09it/s]

jaccard % : 56.71641791044776
cos_sim % : 19.402985074626866


Уже лучше, но все еще плохо. Попробуем вместо максимального значения подобия взять минимальное:

In [37]:
class BetterLeskWithCosSim_InverseArgmax(Lesk):
    
    metric = better_cos_sim

    @classmethod
    def get_definition(cls, word:str, context:str) -> str:

        definitions = cls._get_all_definitions(word)
        metrics: np.ndarray = cls.metric(context, definitions)
        
        #  :o)
        argmax = metrics.argmin()

        return {
            "definition": definitions[argmax],
            "score"     : float(metrics[argmax])
        }

In [38]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

count_results(results)

100%|██████████| 10/10 [00:03<00:00,  3.11it/s]

jaccard % : 56.71641791044776
cos_sim % : 43.28358208955224


Результаты неожиданные. Посмотрим, что посчитается на следующих 20 предложениях

In [39]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    cos_sim_positive_count = 0
    inverse_argmax_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, cos_sim_result, inverse_argmax_result = result

        jaccard_positive_count += jaccard_result == gold
        cos_sim_positive_count += cos_sim_result == gold
        inverse_argmax_positive_count += inverse_argmax_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim_normal % : {cos_sim_positive_count * 100 / total_count}")
    print(f"cos_sim_inverse_argmax % : {inverse_argmax_positive_count * 100 / total_count}")

results = list()


for sentence in tqdm(corpus[10:30]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            inverse_argmax_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                    inverse_argmax_result["definition"]
                )
            )

count_results(results)

100%|██████████| 20/20 [00:17<00:00,  1.18it/s]

jaccard % : 54.30107526881721
cos_sim_normal % : 24.193548387096776
cos_sim_inverse_argmax % : 40.32258064516129


Последний тест -- аргмакс с препроцом и без:

In [40]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    inverse_argmax_positive_count_no_preproc = 0
    inverse_argmax_positive_count_w_preproc = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, inverse_argmax_no_preproc_result, inverse_argmax_w_preproc_result = result

        jaccard_positive_count += jaccard_result == gold
        inverse_argmax_positive_count_no_preproc += inverse_argmax_no_preproc_result == gold
        inverse_argmax_positive_count_w_preproc += inverse_argmax_w_preproc_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim_inverse_argmax no preproc % : {inverse_argmax_positive_count_no_preproc * 100 / total_count}")
    print(f"cos_sim_inverse_argmax + preproc % : {inverse_argmax_positive_count_w_preproc * 100 / total_count}")

results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            inverse_argmax_no_preproc_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            inverse_argmax_w_preproc_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(Preprocessor.preprocess(context))
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    inverse_argmax_no_preproc_result["definition"],
                    inverse_argmax_w_preproc_result["definition"]
                )
            )

count_results(results)

100%|██████████| 10/10 [00:06<00:00,  1.54it/s]

jaccard % : 56.71641791044776
cos_sim_inverse_argmax no preproc % : 43.28358208955224
cos_sim_inverse_argmax + preproc % : 41.791044776119406


Без препроца лучше, дальше весь датасет будем гнать в таком формате: жаккар + препроц, косинусное подобие без препроца. При этом сначала посчитаем жаккара отдельно для всего датасета:

In [41]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result = result

        jaccard_positive_count += jaccard_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")



results = list()

for sentence in tqdm(corpus):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                )
            )

count_results(results)

100%|██████████| 49453/49453 [02:13<00:00, 371.74it/s] 


jaccard % : 43.75961285966163


И теперь посчитаем косинусное подобие отдельно для... ~~30 *вторых*~~ 100 предложений, потому что ~~мой компьютер не выдержит 140 часов расчетов~~ даже на хорошей карточке считает достаточно долго

In [43]:
def count_results(results:list) -> None:

    cos_sim_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, cos_sim_result = result

        cos_sim_positive_count += cos_sim_result == gold

    print(f"cos_sim % : {cos_sim_positive_count * 100 / total_count}")


results = list()

# for sentence in tqdm(corpus[30:60]):
for sentence in tqdm(corpus[:100]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            cos_sim_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    cos_sim_result["definition"],
                )
            )

count_results(results)

100%|██████████| 100/100 [00:35<00:00,  2.79it/s]

cos_sim % : 42.893081761006286


А тут получилось ~~лучше, чем на жаккаре, мб потому, что выборка маленькая.~~ так же. В любом случае, эксперимент заканчиваю.

# Задание 2 (4 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [44]:
import pandas as pd

In [45]:
df = pd.read_csv(r"./data/train.csv", sep="\t")

In [46]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [47]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [48]:
def get_ARI(grouped_df, cluster):

    ARI = []

    for key, _ in grouped_df:
        # вытаскиваем контексты
        texts = grouped_df.get_group(key)['context'].values

        # создаем пустую матрицу для векторов 
        X = np.zeros((len(texts), 768))

        # переводим тексты в векторы и кладем в матрицу
        for i, text in enumerate(texts):
            X[i] = model.encode(text)

        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

        # расчитываем метрику для отдельного слова
        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
        
    print(np.mean(ARI)) # усредненная метрика

In [55]:
kmeans_params = (
    {
        "n_clusters": 8,
        "max_iter": 300,
        "algorithm":"lloyd"
    },
    {
        "n_clusters": 8,
        "max_iter": 300,
        "algorithm":"elkan"
    },
    {
        "n_clusters": 3,
        "max_iter": 200,
        "algorithm":"lloyd"
    },
    {
        "n_clusters": 3,
        "max_iter": 500,
        "algorithm":"lloyd"
    },
    {
        "n_clusters": 3,
        "max_iter": 500,
        "algorithm":"elkan"
    },
    {
        "n_clusters": 6,
        "max_iter": 300,
        "algorithm":"elkan"
    },
)

for params in kmeans_params:
    print(params, end="\t")
    kmeans = KMeans(**params)
    get_ARI(grouped_df, kmeans)


{'n_clusters': 8, 'max_iter': 300, 'algorithm': 'lloyd'}	0.046061661243416444
{'n_clusters': 8, 'max_iter': 300, 'algorithm': 'elkan'}	0.035343829699609156
{'n_clusters': 3, 'max_iter': 200, 'algorithm': 'lloyd'}	0.045589545730551066
{'n_clusters': 3, 'max_iter': 500, 'algorithm': 'lloyd'}	0.026788975182023136
{'n_clusters': 3, 'max_iter': 500, 'algorithm': 'elkan'}	0.10632515799415188
{'n_clusters': 6, 'max_iter': 300, 'algorithm': 'elkan'}	0.04001386670575718


In [60]:
affinity_params = (
    {
        "damping": 0.5,
        "max_iter": 200,
        "affinity":"euclidean",
        "convergence_iter": 15,
    },
    {
        "damping": 0.9,
        "max_iter": 200,
        "affinity":"euclidean",
        "convergence_iter": 15,
    },
    {
        "damping": 0.9,
        "max_iter": 100,
        "affinity":"euclidean",
        "convergence_iter": 45,
    },
    {
        "damping": 0.5,
        "max_iter": 200,
        "affinity":"euclidean",
        "convergence_iter": 20,
    },
    {
        "damping": 0.8,
        "max_iter": 300,
        "affinity":"euclidean",
        "convergence_iter": 25,
    },
    {
        "damping": 0.5,
        "max_iter": 350,
        "affinity":"euclidean",
        "convergence_iter": 35,
    },
)

for params in affinity_params:
    print(params, end="\t")
    affinity = AffinityPropagation(**params)
    get_ARI(grouped_df, affinity)


{'damping': 0.5, 'max_iter': 200, 'affinity': 'euclidean', 'convergence_iter': 15}	0.042740969848549505
{'damping': 0.9, 'max_iter': 200, 'affinity': 'euclidean', 'convergence_iter': 15}	0.05297560306165972
{'damping': 0.9, 'max_iter': 100, 'affinity': 'euclidean', 'convergence_iter': 45}	

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


0.04150923434928373
{'damping': 0.5, 'max_iter': 200, 'affinity': 'euclidean', 'convergence_iter': 20}	0.042740969848549505
{'damping': 0.8, 'max_iter': 300, 'affinity': 'euclidean', 'convergence_iter': 25}	0.04154515818974152
{'damping': 0.5, 'max_iter': 350, 'affinity': 'euclidean', 'convergence_iter': 35}	0.042740969848549505


In [61]:
dbscan_params = (
    {
    "eps": 0.5,
    "min_samples":5,
    },
    {
    "eps": 0.3,
    "min_samples":3,
    },
    {
    "eps": 0.8,
    "min_samples":8,
    },
    {
    "eps": 0.3,
    "min_samples":8,
    },
    {
    "eps": 0.8,
    "min_samples":3,
    },
    {
    "eps": 0.2,
    "min_samples":2,
    },
)

for params in dbscan_params:
    print(params, end="\t")
    dbscan = DBSCAN(**params)
    get_ARI(grouped_df, dbscan)


{'eps': 0.5, 'min_samples': 5}	-0.011271692824715207
{'eps': 0.3, 'min_samples': 3}	-0.031138266011086928
{'eps': 0.8, 'min_samples': 8}	-0.001784882962278153
{'eps': 0.3, 'min_samples': 8}	0.0
{'eps': 0.8, 'min_samples': 3}	-0.001784882962278153
{'eps': 0.2, 'min_samples': 2}	0.007404278375247722


In [63]:
from sklearn.cluster import AgglomerativeClustering

In [65]:
for i in range(2, 7):
    print(f"n_clusters: {i}", end="\t")
    agglomerative = AgglomerativeClustering(i)
    get_ARI(grouped_df, agglomerative)

n_clusters: 2	-0.011976265536517934
n_clusters: 3	0.034738071269679226
n_clusters: 4	0.038617257915792846
n_clusters: 5	0.03379296274962468
n_clusters: 6	0.03957581346650506


##  HDBSCAN

Тот же DBSCAN, но с эвристикой: делается предположение, что эпсилон-окрестность однородна на всей выборке. Автоматически исследуются различные эпсилон-окрестности, выбирается подходящая. Таким образом, появляется возможность собирать кластеры разной плотности на одной выборке.

In [66]:
from sklearn.cluster import HDBSCAN

In [68]:
for i in range(2, 22, 4):
    print(f"n_clusters: {i}", end="\t") 
    hdbscan = HDBSCAN(min_samples=i)
    get_ARI(grouped_df, hdbscan)

n_clusters: 2	0.008236037742083282
n_clusters: 6	-0.013894256669353344
n_clusters: 10	-0.02876603736237084
n_clusters: 14	0.0
n_clusters: 18	0.0
